In [1]:
from pyrocko import util, model, io, trace, moment_tensor, gmtpy
from pyrocko import pz
from pyrocko import orthodrome as od
from pyrocko.io import quakeml
from pyrocko.io import stationxml as fdsn
from pyrocko.client import catalog
from pyrocko.automap import Map
import pyrocko.moment_tensor as pmt
from seiscloud import plot as scp
from seiscloud import cluster as scc
import numpy as num
import os, sys, re, math, shutil
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
from matplotlib import dates
import datetime
import urllib.request
from pyrocko.plot.gmtpy import GMT

In [2]:
workdir='../'
catdir =  os.path.join(workdir,'CAT')

## GOSSIP: Events where either lat, long, depth or mag are not present

In [ ]:
catname = os.path.join(catdir, 'catologue_flegrei_GOSSIP_complete.pf')

cat_c = model.load_events(catname)
print('Number of events:', len(cat_c))

In [ ]:
magnitude_missed=[]
time=[]
for ev in cat_c:
    if ev.magnitude==None or ev.lat==None or ev.lon==None or ev.depth==None:
        magnitude_missed.append(ev.magnitude)
        time.append(ev.time)

eq_dates = [datetime.datetime.fromtimestamp(t) for t in time]

fig, axs = plt.subplots(1, 1, figsize=(17,6), sharex=True)
axs.scatter(eq_dates,magnitude_missed, s=5, color='green')
axs.set_ylabel('Magnitude')
axs.grid(True)
axs.set_xlabel('Time')


max=0
for mag in magnitude_missed:
    if mag != None:
        if mag>max:
            max=mag
print('max magnitude lost:',max)



## new catalogue with events over a certain magnitude

### GOSSIP

In [ ]:
catname = os.path.join(catdir, 'catologue_flegrei_GOSSIP.pf')

cat = model.load_events(catname)
print('Number of events:', len(cat))

In [ ]:
events_mag = []

minimum_magnitude=2.5

for ev in cat:
    if ev.magnitude>=minimum_magnitude :
        events_mag.append(model.Event(name=ev.name, time=ev.time,
                                lat=ev.lat, lon=ev.lon,
                                depth=ev.depth, magnitude=ev.magnitude))
events_mag.sort(key=lambda x: x.time, reverse=False)

catname = os.path.join(catdir, 'catologue_flegrei_GOSSIP_mag2_5.pf')

model.dump_events(events_mag, catname)
print('Number of events:', len(events_mag))

### INGV

In [ ]:
catname = os.path.join(catdir, 'catologue_flegrei_INGV.pf')

cat = model.load_events(catname)
print('Number of events:', len(cat))

In [ ]:
events_mag = []

minimum_magnitude=2.5

for ev in cat:
    if ev.magnitude>=minimum_magnitude :
        events_mag.append(model.Event(name=ev.name, time=ev.time,
                                lat=ev.lat, lon=ev.lon,
                                depth=ev.depth, magnitude=ev.magnitude))
events_mag.sort(key=lambda x: x.time, reverse=False)

catname = os.path.join(catdir, 'catologue_flegrei_INGV_mag2_5.pf')

model.dump_events(events_mag, catname)
print('Number of events:', len(events_mag))

# compare the two catalogues

In [10]:
catname = os.path.join(catdir, 'catologue_flegrei_GOSSIP_mag2_5.pf')

catalogue_G = model.load_events(catname)
print('Number of events GOSSIP:', len(catalogue_G))

catname = os.path.join(catdir, 'catologue_flegrei_INGV_mag2_5.pf')

catalogue_I = model.load_events(catname)
print('Number of events INGV:', len(catalogue_I))

Number of events GOSSIP: 51
Number of events INGV: 43


In [ ]:
id_catalogues=[]

for ind,ev_G in enumerate(catalogue_G):
    
    origin_time = ev_G.time
    time_min= origin_time - 5
    time_max= origin_time + 5

    id_catalogues.append([])
    id_catalogues[ind].append( ev_G.name )

    for ev_I in catalogue_I:
        if ev_I.time>time_min and ev_I.time<time_max:
            id_catalogues[ind].append( ev_I.name )
    

id_catalogues

In [ ]:
id_catalogues2=[]

for ind,ev_I in enumerate(catalogue_I):
    
    origin_time = ev_I.time
    time_min= origin_time - 5
    time_max= origin_time + 5

    id_catalogues2.append([])
    id_catalogues2[ind].append( ev_I.name )

    for ev_G in catalogue_G:
        if ev_G.time>time_min and ev_G.time<time_max:
            id_catalogues2[ind].append( ev_G.name )
    

id_catalogues2

In [14]:
events = []

#create a new catalogue with all the events from GOSSIP
for ev_G in catalogue_G:
#    events.append(ev_G)
#    ev_G.tags = ['GOSSIP_id:'+ str(ev_G.name) + ' mag:' + str(ev_G.magnitude)]
#    strtime = util.time_to_str(ev_G.time)
#    ev_G.name = 'flegrei_' + strtime[0:4] +'_'+ strtime[5:7]+'_'+strtime[8:10]+'_'+strtime[11:13]+'_'+strtime[14:16]+'_'+strtime[17:19]
    tag = ['GOSSIP_id:'+ str(ev_G.name) + ', mag:' + str(ev_G.magnitude)]
    strtime = util.time_to_str(ev_G.time)
    event_name = 'flegrei_' + strtime[0:4] +'_'+ strtime[5:7]+'_'+strtime[8:10]+'_'+strtime[11:13]+'_'+strtime[14:16]+'_'+strtime[17:19]
    events.append(model.Event(name=event_name, time=ev_G.time,
                            lat=ev_G.lat, lon=ev_G.lon,
                            depth=ev_G.depth,tags=tag))


events.sort(key=lambda x: x.time, reverse=False)
catname = os.path.join(catdir, 'catologue_flegrei_mag2_5.pf')
model.dump_events(events, catname)

events=model.load_events(catname)

#add the tags on the common events with INGV
ev_to_remove=[]

for ind,ev_G in enumerate(events):
    time_min= ev_G.time -5
    time_max= ev_G.time +5
    count=0
    for ev_I in catalogue_I:
        if ev_I.time>time_min and ev_I.time<time_max:
            ev_G.tags.append('INGV_id:'+ str(ev_I.name) + ', mag:' + str(ev_I.magnitude))
            count+=1
    if count!=1:
        ev_to_remove.append(ind)

for ind in reversed(ev_to_remove):
    del events[ind]

events.sort(key=lambda x: x.time, reverse=False)


catname = os.path.join(catdir, 'catologue_flegrei_mag2_5.pf')
model.dump_events(events, catname)
print('Number of events of merged catalogues:', len(events))

Number of events of merged catalogues: 38


In [12]:
for ev in events:
    print(ev.tags)

['GOSSIP_id:4217', 'mag:2.5', 'INGV_id:6142151, mag:2.5']
['GOSSIP_id:7102', 'mag:3.47', 'INGV_id:30205901, mag:2.7']
['GOSSIP_id:14725', 'mag:3.57', 'INGV_id:30350681, mag:3.0']
['GOSSIP_id:16351', 'mag:2.51', 'INGV_id:32166291, mag:2.7']
['GOSSIP_id:17434', 'mag:2.67', 'INGV_id:33361881, mag:2.7']
['GOSSIP_id:17732', 'mag:2.53', 'INGV_id:33524091, mag:2.6']
['GOSSIP_id:23194', 'mag:2.59', 'INGV_id:33946951, mag:2.5']
['GOSSIP_id:23338', 'mag:2.97', 'INGV_id:34043501, mag:3.0']
['GOSSIP_id:23716', 'mag:2.6', 'INGV_id:34240361, mag:2.6']
['GOSSIP_id:23904', 'mag:2.83', 'INGV_id:34351831, mag:2.8']
['GOSSIP_id:24146', 'mag:2.59', 'INGV_id:34448221, mag:2.6']
['GOSSIP_id:24479', 'mag:2.92', 'INGV_id:34628851, mag:2.7']
['GOSSIP_id:24645', 'mag:2.63', 'INGV_id:34689631, mag:2.8']
['GOSSIP_id:24688', 'mag:2.71', 'INGV_id:34689631, mag:2.8']
['GOSSIP_id:25202', 'mag:3.48', 'INGV_id:34919151, mag:3.4']
['GOSSIP_id:25214', 'mag:2.69', 'INGV_id:34928261, mag:2.8']
['GOSSIP_id:26132', 'mag:3.56

In [13]:
for ev in events:
    print(ev.name)

flegrei_2015_10_07_09_10_50
flegrei_2022_03_16_14_14_34
flegrei_2022_03_29_17_45_32
flegrei_2022_07_30_03_27_48
flegrei_2022_11_12_21_37_54
flegrei_2022_11_30_02_34_39
flegrei_2023_01_19_08_44_30
flegrei_2023_02_05_00_45_36
flegrei_2023_03_04_13_15_20
flegrei_2023_03_13_22_40_53
flegrei_2023_03_24_05_24_43
flegrei_2023_04_10_03_54_45
flegrei_2023_04_15_05_54_37
flegrei_2023_04_15_05_54_40
flegrei_2023_05_08_02_28_34
flegrei_2023_05_08_22_33_17
flegrei_2023_06_11_06_44_25
flegrei_2023_06_15_15_39_40
flegrei_2023_08_18_00_01_28
flegrei_2023_08_18_04_09_59
flegrei_2023_08_18_04_10_30
flegrei_2023_08_18_04_18_05
flegrei_2023_08_18_04_22_49
flegrei_2023_08_27_21_14_23
flegrei_2023_09_07_17_45_28
flegrei_2023_09_12_02_28_50
flegrei_2023_09_20_22_54_29
flegrei_2023_09_22_09_01_59
flegrei_2023_09_24_15_35_02
flegrei_2023_09_26_07_10_29
flegrei_2023_09_27_01_35_34
flegrei_2023_09_27_09_16_18
flegrei_2023_09_29_21_57_58
flegrei_2023_10_01_13_33_09
flegrei_2023_10_02_20_08_26
flegrei_2023_10_04_0

### Load

In [ ]:
catname = os.path.join(catdir, 'catologue_flegrei_mag2_5.pf')

cat = model.load_events(catname)
print('Number of events:', len(cat))